In [61]:
import plotly.express as px

import numpy as np 
import pandas as pd
import seaborn as sns

from sklearn import datasets
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.utils import check_random_state
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy.stats import fisher_exact


from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import sklearn.cluster as cluster
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from hdbscan import HDBSCAN
from IPython.display import Image
import pydotplus
from six import StringIO
from imblearn.under_sampling import RandomUnderSampler
from random import sample
from random import seed

import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

from umap import UMAP

%matplotlib inline

seed(24)


## 1. Load PSD expression data

In [62]:
rna = pd.read_csv("geneexpression_labelled_region_ALLENcounts.csv")


In [63]:
tfr = pd.read_csv("Allen_labelledcounts_ALLGENES.csv", chunksize=1024, iterator=True)
rna2 = pd.concat(tfr, ignore_index=True)

samples2= rna2['sample_name']
labels2 = rna2['subclass_label']
del rna2['sample_name']
del rna2['subclass_label']


In [64]:
# Only 20% of DG cells where selected on the all transcriptome dataframe due to memory issues. 
# We will select the same cells for PSD genes dataframe

#matching_samples = samples1.isin(samples2)


rna = rna[rna['Row.names'].isin(samples2)]
rna.reset_index()
#labels1= labels1[matching_samples]


samples1= rna['Row.names']
labels1 = rna['subclass_label']

del rna['Row.names']
del rna['subclass_label']


### Exploratory analysis

In [65]:
label= sorted(list(set(labels1)))

print("PSD genes only")
print()
print("Number of genes: ", len(rna.columns))
print("Number of cells: ", len(rna))
print("Number of labels: ", len(labels1))

 
print()
print("Occurrences of classes")
for lab in label:
    print(lab + " : " + str(list(labels1.values).count(lab)))

print()
print("-------------------------------------")
print()
print("All transcriptome")

print()

print("Number of genes: ", len(rna2.columns))
print("Number of cells: ", len(rna2))
print("Number of labels: ", len(labels2))

print()
 
print("Occurrences of classes")
for lab in label:
    print(lab + " : " + str(list(labels2.values).count(lab)))
    


PSD genes only

Number of genes:  4074
Number of cells:  41506
Number of labels:  41506

Occurrences of classes
CA1 : 12767
CA1-ProS : 3355
CA2 : 143
CA3 : 1899
CT SUB : 5414
DG : 11664
NP SUB : 1885
SUB-ProS : 4379

-------------------------------------

All transcriptome

Number of genes:  31053
Number of cells:  41506
Number of labels:  41506

Occurrences of classes
CA1 : 12767
CA1-ProS : 3355
CA2 : 143
CA3 : 1899
CT SUB : 5414
DG : 11664
NP SUB : 1885
SUB-ProS : 4379


# Random Forest models

First, we use GridSearch on PSD dataset to look for the best parameters

In [ ]:
max_depth_l = list(range(5,13))
n_estimators_l= [50,100,200]
param_d = dict(max_depth=max_depth_l, n_estimators= n_estimators_l)

X_train,X_test,y_train,y_test = train_test_split(rna, labels1,random_state=24,train_size=0.8)

model_psd =  RandomForestClassifier()
grid_search = GridSearchCV(model_psd, param_d, cv=4)
grid_fit=grid_search.fit(X_train, y_train)


score_pd=pd.DataFrame(grid_search.cv_results_)
scores_2d=[]
i=0
for c in range(len(max_depth_l)):
    scores_mat=[]
    for g in range(len(n_estimators_l)):
        mean_score= grid_search.cv_results_["mean_test_score"][i]
        scores_mat.append(mean_score)
        i+=1
    scores_2d.append(scores_mat)

fig, ax = plt.subplots(figsize=(13,10)) 
sns.heatmap(scores_2d,ax=ax, annot=True)
ax.set_xticklabels(n_estimators_l)
ax.set_yticklabels(max_depth_l)
ax.set_xlabel("N estimators")
ax.set_ylabel("Max Depth")
fig.savefig('../MODELRF_params.svg')
plt.show()

In [ ]:
fig.savefig('../MODELRF_params.svg')

Best parametes n_estimators 200 max_depth 12

### Random Forest model (PSD genes) with best parameters

In [ ]:
model_psd =  RandomForestClassifier(n_estimators=200, max_depth=12, random_state=24)
model_psd.fit(X_train, y_train)
y_pred_train = model_psd.predict(X_train)
y_pred_test = model_psd.predict(X_test)



In [ ]:
accuracy_train = accuracy_score(y_train, y_pred_train)
conf_mat_train = confusion_matrix(y_train, y_pred_train, labels= label)
accuracy_test = accuracy_score(y_test, y_pred_test)
conf_mat_test = confusion_matrix(y_test, y_pred_test, labels= label)
confusion_matrix_norm = conf_mat_test.astype('float') / conf_mat_test.sum(axis=1)[:, np.newaxis]

print("Train Set")
print("Accuracy: ", accuracy_train)
print(conf_mat_train)
print("Test Set")
print("Accuracy: ", accuracy_test)
print(conf_mat_test)

In [ ]:
accuracy_test = pd.DataFrame(confusion_matrix_norm,index=label, columns=label)
accuracy_test = accuracy_test.reindex(sorted(accuracy_test.columns), axis=1)

fig, ax = plt.subplots(figsize=(13,10)) 

sns.heatmap(np.array(accuracy_test),ax=ax, annot=True, vmin=0, vmax=1, fmt='.4f', cmap="Oranges", annot_kws={"fontsize":14})
ax.set_xticklabels(label)
ax.set_yticklabels(label)
ax.set_xlabel("Predicted Class")
ax.set_ylabel("True Class")
fig.savefig('../MODELRFPSD_confussion_matrix_test.svg')
plt.show()

### Analysis of feature importances

In [ ]:
importances = model_psd.feature_importances_
std = np.std([tree.feature_importances_ for tree in model_psd.estimators_], axis=0)
forest_importances = pd.Series(importances, index=X_train.columns)



#plt.hist(forest_importances)

#print(forest_importances.median())
print(forest_importances.sort_values(ascending=False)[0:20])
forest_importances.sort_values(ascending=False).to_csv("../feats_importances_PSDGENES.csv")
psdmodel_importances= forest_importances.sort_values(ascending=False)


### Graph of cumulative feature importances


In [ ]:
psdmodel_cumsum_importances=0
psdmodel_cumsum=[]
totaln= 1000
#totaln= len(psdmodel_importances) 

for i in range(totaln):
    psdmodel_cumsum_importances += psdmodel_importances[i]
    psdmodel_cumsum.append(psdmodel_cumsum_importances)
    
fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
ax.plot(list(range(1,totaln+1)), psdmodel_cumsum, color="b", label="Total")
#ax.plot(list(range(1,totaln+1)), lpsd_cumsum, color="g", label="Genes from PSD")
#ax.plot(list(range(1,totaln+1)), lnopsd_cumsum, color="y", label="non-PSD genes" )
#ax.legend()
ax.set_xlabel("Number of genes")
ax.set_ylabel("Feature importance")
#fig.savefig('../importanceS_PSDGENES_1000genes.svg')   # save the figure to file
#plt.close(fig) 
plt.show()

The number of genes where the curve start to saturate is around 500 with only PSD genes

## Random Forest model with whole transcriptome

We will use the same parameters as before

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(rna2, labels2,random_state=24,train_size=0.8)


In [ ]:
model_allgenes =  RandomForestClassifier(n_estimators=200, max_depth=12, random_state=24)
model_allgenes.fit(X_train, y_train)
y_pred_train = model_allgenes.predict(X_train)
y_pred_test = model_allgenes.predict(X_test)



In [ ]:
label = sorted(list(set(y_train)))

accuracy_train = accuracy_score(y_train, y_pred_train)
conf_mat_train = confusion_matrix(y_train, y_pred_train, labels= label)
accuracy_test = accuracy_score(y_test, y_pred_test)
conf_mat_test = confusion_matrix(y_test, y_pred_test, labels= label)

confusion_matrix_norm = conf_mat_test.astype('float') / conf_mat_test.sum(axis=1)[:, np.newaxis]

print("Train Set")
print("Accuracy: ", accuracy_train)
print(conf_mat_train)
print()
print("Test Set")
print("Accuracy: ", accuracy_test)
print(confusion_matrix_norm)

In [ ]:
accuracy_test = pd.DataFrame(confusion_matrix_norm,index=label, columns=label)
accuracy_test = accuracy_test.reindex(sorted(accuracy_test.columns), axis=1)

fig, ax = plt.subplots(figsize=(13,10)) 


sns.heatmap(np.array(accuracy_test),ax=ax, annot=True, vmin=0, vmax=1, fmt='.4f', cmap="Oranges", annot_kws={"fontsize":14} )
ax.set_xticklabels(label)
ax.set_yticklabels(label)
ax.set_xlabel("Predicted Class")
ax.set_ylabel("True Class")

fig.savefig('../MODELRF_confussion_matrix_test_wholetransc.svg')
plt.show()

In [ ]:
importances = model_allgenes.feature_importances_
std = np.std([tree.feature_importances_ for tree in model_allgenes.estimators_], axis=0)
forest_importances = pd.Series(importances, index=X_train.columns)


#plt.hist(forest_importances)

#print(forest_importances.median())
forest_importances.sort_values(ascending=False).to_csv("../feats_importances_ALLGENES.csv")

## Load feature importances (PSD and ALL TRANSCRIPTOME)

In [66]:
feature_importances_psd = pd.read_csv("../feats_importances_PSDGENES.csv")
feature_importances_alltc= pd.read_csv("../feats_importances_ALLGENES.csv")
non_PSD_genes = list(set(feature_importances_psd['Unnamed: 0'].values) - set(feature_importances_alltc['Unnamed: 0'].values))
feature_importances_nonPSD = feature_importances_alltc[feature_importances_alltc['Unnamed: 0'].isin(non_PSD_genes)].sort_values(by="0",ascending=False).reset_index(drop=True)
importances_allgenes_psd= feature_importances_psd["0"]
importances_allgenes_alltc= feature_importances_alltc["0"]


### Random Forest model with 500 more important genes of the PSD

## PSD genes vs non-PSD genes

### Top 1000 PSD vs no-PSD

In [ ]:
importants_1000 = feature_importances_alltc['Unnamed: 0'][0:1000]
y = np.array([sum(rna.columns.isin(importants_1000)), 1000 - sum(rna.columns.isin(importants_1000))])
mylabels = ["PSD", "Non_PSD"]

fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
ax.pie(y, labels = mylabels)
fig.savefig('PSDvsNonPSD_pie.svg')   # save the figure to file
plt.show()
plt.close(fig) 

In [ ]:
cumsum_importances=0
l_cumsum=[]
cumsum_psdimportances=0
cumsum_nopsdimportances=0
lpsd_cumsum=[]
lnopsd_cumsum=[]
totaln= 10000
#totaln= len(feature_importances) 

for i in range(totaln):
    cumsum_importances += feature_importances_alltc["0"][i]
    genename= feature_importances_alltc["Unnamed: 0"][i]
    
    if genename in psd_prots_l:
        cumsum_psdimportances+= feature_importances_alltc["0"][i]
    else:
        cumsum_nopsdimportances+= feature_importances_alltc["0"][i]
        
    l_cumsum.append(cumsum_importances)
    lpsd_cumsum.append(cumsum_psdimportances)
    lnopsd_cumsum.append(cumsum_nopsdimportances)

fig, ax = plt.subplots( nrows=1, ncols=1 )  # create figure & 1 axis
ax.plot(list(range(1,totaln+1)), l_cumsum, color="b", label="Total")
#ax.plot(list(range(1,totaln+1)), lpsd_cumsum, color="g", label="Genes from PSD")
#ax.plot(list(range(1,totaln+1)), lnopsd_cumsum, color="y", label="non-PSD genes" )
#ax.legend()
ax.set_xlabel("Number of genes")
ax.set_ylabel("Feature importance")
fig.savefig('../feats_importancesRF_ALLGENES_only2000genes.svg')   # save the figure to file
#plt.close(fig) 
plt.show()

It shows that the optimal number of genes to take into account in the classification is around 1000 if we compare the whole transcriptome

# Chi Squared tests

### Chi Squared PSD vs noPSD

In [53]:
psd_genes = feature_importances_psd["Unnamed: 0"]

num_psdgenes_present = len(psd_genes)
num_psdgenes_important= len(list(psd_genes[psd_genes.isin(feature_importances_alltc["Unnamed: 0"][0:1000])].values))

num_psdgenes_not_important= num_psdgenes_present - num_psdgenes_important

rest_important = 1000 - num_psdgenes_important
rest_not_important = len(feature_importances_alltc) - (1000 + num_psdgenes_not_important)

table = [[num_psdgenes_important, num_psdgenes_not_important],[rest_important, rest_not_important ]]
print(table)

stat, p = fisher_exact(table, alternative ="greater")
print(stat,p)


[[518, 3556], [482, 26497]]
8.00788218381416 1.3022186923268363e-197


# UMAP representation

In [ ]:
print(len(list(set(list(rna2.columns)) - set(list(rna.columns)))))
label = sorted(list(set(labels1)))


### UMAP PSD genes

In [ ]:
rna_umap = UMAP(random_state=24, n_neighbors=15, min_dist= 0.1)

# Fit UMAP and extract latent vars 1-2
embedding_psd_a = pd.DataFrame(rna_umap.fit_transform(rna), columns = ['UMAP1','UMAP2'])


# Produce sns.scatterplot and pass metadata.subclasses as color
sns_plot = sns.scatterplot(x='UMAP1', y='UMAP2', data=embedding_psd_a, hue=labels1.values, hue_order=label,alpha=.1, linewidth=0, s=1)

# Adjust legend
sns_plot.legend(loc='center left', bbox_to_anchor=(1, .5))
# Save PNG
plt.show()
sns_plot.figure.savefig('umap_PSD_all.svg', bbox_inches='tight')